In [1]:
import boto3
import tempfile

In [2]:
region = 'us-east-1'
cf_stack_name = 'Playground1Stack'

In [3]:
# Get BucketName from CloudFormation Stack output
def get_bucket_name(stack_name):
    cf_client = boto3.client('cloudformation', region_name=region)
    response = cf_client.describe_stacks(StackName=stack_name)
    for output in response['Stacks'][0]['Outputs']:
        if output['OutputKey'] == 'BucketName':
            return output['OutputValue']

    raise Exception('BucketName not found in CloudFormation Stack output')

In [4]:
bucket_name = get_bucket_name(cf_stack_name)

# Create S3 client
s3 = boto3.client('s3', region_name=region)
bucket = boto3.resource('s3').Bucket(bucket_name)

In [5]:
# Upload a dummy object
with tempfile.TemporaryFile() as tmp:
    tmp.write(b'Hello, world!')
    tmp.seek(0)
    bucket.upload_fileobj(tmp, 'dummy.txt')

In [6]:
# List objects in the bucket
for obj in bucket.objects.all():
    print(obj.key)

dummy.txt


In [ ]:
# Retrieve and print the content of the dummy object
obj = bucket.Object('dummy.txt')
print(obj.get()['Body'].read().decode('utf-8'))

Hello, world!


In [8]:
# Delete the dummy object
obj.delete()

{'ResponseMetadata': {'RequestId': '6NNQ3N87VB3G5QTV',
  'HostId': 'IgmW32P09he74zDxTu0cJcDPjRmRbEDehraHj3c0qXb+YpdnMs5BZYQ/4/VE9pW+veKySLjgf9xpHMApKRxOrI0urUX01Y47wKwHaNQS3A4=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'IgmW32P09he74zDxTu0cJcDPjRmRbEDehraHj3c0qXb+YpdnMs5BZYQ/4/VE9pW+veKySLjgf9xpHMApKRxOrI0urUX01Y47wKwHaNQS3A4=',
   'x-amz-request-id': '6NNQ3N87VB3G5QTV',
   'date': 'Tue, 19 Nov 2024 22:04:36 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [9]:
# List objects in the bucket
for obj in bucket.objects.all():
    print(obj.key)

In [10]:
dynamodb_table_prefix = 'Playground1Table-MyTable'

# Create DynamoDB client
dynamodb = boto3.client('dynamodb', region_name=region)

In [12]:
# Find the table name
table_name = None
response = dynamodb.list_tables()
for tn in response['TableNames']:
    if tn.startswith(dynamodb_table_prefix):
        print('Found table:', tn)
        table_name = tn
        break

if table_name is None:
    raise Exception('Table not found')

ClientError: An error occurred (AccessDeniedException) when calling the ListTables operation: User: arn:aws:sts::404449475211:assumed-role/Playground1Stack-EC2InstanceRole0BAA8CDD-gijpoD4ZOqNC/i-0599c09eb6cd71ce6 is not authorized to perform: dynamodb:ListTables on resource: arn:aws:dynamodb:us-east-1:404449475211:table/* because no identity-based policy allows the dynamodb:ListTables action

In [ ]:
students = [
    {
        'id': '1',
        'name': 'Alice',
        'class': 'A',
        'score': 85
    },
    {
        'id': '2',
        'name': 'Bob',
        'class': 'B',
        'score': 70
    },
    {
        'id': '3',
        'name': 'Charlie',
        'class': 'A',
        'score': 60
    },
    {
        'id': '4',
        'name': 'David',
        'class': 'B',
        'score': 90
    },
    {
        'id': '5',
        'name': 'Eve',
        'class': 'A',
        'score': 95
    }
]

In [ ]:
# Add items to the table
for student in students:
    dynamodb.put_item(
        TableName=table_name,
        Item={
            'id': {'S': student['id']},
            'name': {'S': student['name']},
            'class': {'S': student['class']},
            'score': {'N': str(student['score'])}
        }
    )

# Scan the table to list elements with score <= 70
response = dynamodb.scan(
    TableName=table_name,
    FilterExpression='score <= :score',
    ExpressionAttributeValues={':score': {'N': '70'}}
)

for item in response['Items']:
    print(item)